In [19]:
carpeta = "../plantvillage_dataset/grayscale/"

Codigo para entrenar el modelo a blanco y negro:

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras import Sequential, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Configurar rutas y parámetros
dataset_path = carpeta  # Cambia esto a la ruta de tu dataset
target_size = (128, 128)  # Redimensionar a 128x128
batch_size = 32
epochs = 10

# 2. Preprocesamiento y aumento de datos
datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalización de píxeles
    rotation_range=20,  # Rotación aleatoria
    width_shift_range=0.2,  # Desplazamiento horizontal aleatorio
    height_shift_range=0.2,  # Desplazamiento vertical aleatorio
    horizontal_flip=True,  # Volteo horizontal aleatorio
    validation_split=0.2  # 20% de los datos para validación
)

# 3. Cargar las imágenes de las 3 clases
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=target_size,
    color_mode='grayscalegrayscale',  # Cargar en escala de grises
    batch_size=batch_size,
    class_mode='categorical',  # Clasificación multiclase
    classes=['Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Septoria_leaf_spot'],  # Tres clases
    subset='training'  # Conjunto de entrenamiento
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=target_size,
    color_mode='grayscale',  # Cargar en escala de grises
    batch_size=batch_size,
    class_mode='categorical',  # Clasificación multiclase
    classes=['Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Septoria_leaf_spot'],  # Tres clases
    subset='validation'  # Conjunto de validación
)

# 4. Construir el modelo
model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')  # Capa de salida para 3 clases
])

# 5. Compilar el modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Pérdida para clasificación multiclase
              metrics=['accuracy'])

# 6. Resumen del modelo
model.summary()

# 7. Entrenar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

# 8. Evaluar el modelo
loss, accuracy = model.evaluate(validation_generator)
print(f"Pérdida en validación: {loss}")
print(f"Precisión en validación: {accuracy}")

# 9. Guardar el modelo
model.save("modelo_tomate_3_clases.keras")

Codigo para graficar la confianza:

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='Precisión en entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión en validación')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()
plt.show()

# Gráfico de pérdida
plt.plot(history.history['loss'], label='Pérdida en entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida en validación')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

Codigo para ejecutar el modelo en blanco y negro:

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import pyautogui
from PIL import Image

# 1. Cargar el modelo entrenado
model = tf.keras.models.load_model("modelo_tomate_3_clases_escala_grises.keras")  # Asegúrate de que el modelo esté en la ruta correcta

# 2. Definir el tamaño de la región de la pantalla a capturar
screen_width, screen_height = 400, 400  # Resolución de 400x400

# 3. Definir las etiquetas de las clases
class_names = ['Bueno', "Late", 'Septoria']

# 4. Función para preprocesar la imagen
def preprocess_image(image):
    # Convertir a escala de grises
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Redimensionar a 256x256 (tamaño esperado por el modelo)
    resized_image = cv2.resize(gray_image, (128, 128))
    # Normalizar los valores de píxeles
    normalized_image = resized_image / 255.0
    # Expandir dimensiones para que coincida con la entrada del modelo (256, 256, 1)
    input_image = np.expand_dims(normalized_image, axis=-1)
    input_image = np.expand_dims(input_image, axis=0)  # Añadir dimensión del batch
    return input_image

# 5. Función para predecir la clase y la confianza
def predict_plant(image):
    # Preprocesar la imagen
    input_image = preprocess_image(image)
    # Realizar la predicción
    predictions = model.predict(input_image)
    # Obtener la clase predicha y su confianza
    predicted_class_index = np.argmax(predictions, axis=1)[0]  # Índice de la clase predicha
    predicted_class_name = class_names[predicted_class_index]  # Nombre de la clase predicha
    confidence = np.max(predictions)  # Confianza de la predicción (valor máximo del array de predicciones)
    return predicted_class_name, confidence

# 6. Capturar la pantalla en tiempo real y realizar predicciones
print("Iniciando detección en tiempo real...")
while True:
    # Capturar la pantalla
    screenshot = pyautogui.screenshot(region=(200, 300, screen_width, screen_height))
    screenshot = np.array(screenshot)  # Convertir a un array de NumPy
    screenshot = cv2.cvtColor(screenshot, cv2.COLOR_RGB2BGR)  # Convertir a BGR (OpenCV usa BGR por defecto)

    # Realizar la predicción
    predicted_class_name, confidence = predict_plant(screenshot)

    # Mostrar el resultado en la pantalla
    label = f"{predicted_class_name} ({confidence * 100:.2f}%)"  # Formato: "Clase (Confianza %)"

    # Mostrar el texto en la pantalla
    cv2.putText(screenshot, f"Estado: {label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(screenshot, f"Confianza: {confidence:.2f}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Mostrar la imagen en una ventana
    cv2.imshow("Detección en tiempo real", screenshot)

    # Salir del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cerrar la ventana
cv2.destroyAllWindows()

Codigo para ejecutar modelos a color con trakeo de hojas:

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import pyautogui
from PIL import Image

# 1. Cargar el modelo entrenado
model = tf.keras.models.load_model("modelo_tomate_3_color2.keras")  # Asegúrate de que el modelo esté en la ruta correcta

# 2. Definir el tamaño de la región de la pantalla a capturar
screen_width, screen_height = 400, 400  # Resolución de 400x400

# 3. Definir las etiquetas de las clases
class_names = ['Bueno', "Late", 'Septoria']

# 4. Función para preprocesar la imagen (ahora en color)
def preprocess_image(image):
    # Redimensionar a 128x128 (tamaño esperado por el modelo)
    resized_image = cv2.resize(image, (128, 128))
    # Normalizar los valores de píxeles
    normalized_image = resized_image / 255.0
    # Expandir dimensiones para que coincida con la entrada del modelo (128, 128, 3)
    input_image = np.expand_dims(normalized_image, axis=0)  # Añadir dimensión del batch
    return input_image

# 5. Función para detectar la hoja en la imagen
def detect_leaf(image):
    # Convertir la imagen a espacio de color HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Definir un rango de color verde para detectar la hoja
    lower_green = np.array([35, 50, 50])  # Límite inferior del verde
    upper_green = np.array([85, 255, 255])  # Límite superior del verde
    
    # Crear una máscara para el color verde
    mask = cv2.inRange(hsv_image, lower_green, upper_green)
    
    # Encontrar contornos en la máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Si se detectan contornos, encontrar el más grande (la hoja)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        # Obtener las coordenadas del rectángulo que encierra la hoja
        x, y, w, h = cv2.boundingRect(largest_contour)
        return (x, y, w, h)
    else:
        return None

# 6. Función para predecir la clase y la confianza
def predict_plant(image):
    # Preprocesar la imagen
    input_image = preprocess_image(image)
    # Realizar la predicción
    predictions = model.predict(input_image)
    # Obtener la clase predicha y su confianza
    predicted_class_index = np.argmax(predictions, axis=1)[0]  # Índice de la clase predicha
    predicted_class_name = class_names[predicted_class_index]  # Nombre de la clase predicha
    confidence = np.max(predictions)  # Confianza de la predicción (valor máximo del array de predicciones)
    return predicted_class_name, confidence

# 7. Capturar la pantalla en tiempo real y realizar predicciones
print("Iniciando detección en tiempo real...")
while True:
    # Capturar la pantalla
    screenshot = pyautogui.screenshot(region=(200, 300, screen_width, screen_height))
    screenshot = np.array(screenshot)  # Convertir a un array de NumPy
    screenshot = cv2.cvtColor(screenshot, cv2.COLOR_RGB2BGR)  # Convertir a BGR (OpenCV usa BGR por defecto)

    # Detectar la hoja en la imagen
    leaf_box = detect_leaf(screenshot)
    
    # Si se detecta la hoja, dibujar un rectángulo naranja alrededor
    if leaf_box:
        x, y, w, h = leaf_box
        cv2.rectangle(screenshot, (x, y), (x + w, y + h), (0, 165, 255), 2)  # Rectángulo naranja

    # Realizar la predicción
    predicted_class_name, confidence = predict_plant(screenshot)

    # Mostrar el resultado en la pantalla
    label = f"{predicted_class_name} ({confidence * 100:.2f}%)"  # Formato: "Clase (Confianza %)"

    # Mostrar el texto en la pantalla
    cv2.putText(screenshot, f"Estado: {label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(screenshot, f"Confianza: {confidence:.2f}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Mostrar la imagen en una ventana
    cv2.imshow("Detección en tiempo real", screenshot)

    # Salir del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cerrar la ventana
cv2.destroyAllWindows()